In [2]:
import json
from tqdm import tqdm

# 1. 데이터 로드
with open("data/review_0503.json", "r", encoding="utf-8") as f:
    reviews = [json.loads(line) for line in f]

with open("data/user_0503.json", "r", encoding="utf-8") as f:
    users = [json.loads(line) for line in f]

with open("data/business.json", "r", encoding="utf-8") as f:
    businesses = [json.loads(line) for line in f]


In [4]:
import json
import pandas as pd
from tqdm import tqdm


# 2. DataFrame으로 변환
df_reviews = pd.DataFrame(reviews)
df_users = pd.DataFrame(users)
df_businesses = pd.DataFrame(businesses)

# 3. 정보 출력 함수
def print_info(df, name):
    print(f"\n📌 {name} 정보")
    print("-" * 40)
    print(df.info())
    print(df.head(2))

# 4. 각 파일의 정보 출력
print_info(df_reviews, "리뷰 데이터")
print_info(df_users, "사용자 데이터")
print_info(df_businesses, "음식점 데이터")



📌 리뷰 데이터 정보
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676701 entries, 0 to 676700
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   review_id     676701 non-null  object
 1   user_id       676701 non-null  object
 2   business_id   676701 non-null  object
 3   stars         676701 non-null  int64 
 4   useful        676701 non-null  int64 
 5   date          676701 non-null  int64 
 6   sentiment     676701 non-null  object
 7   cleaned_text  676701 non-null  object
 8   token_length  676701 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 46.5+ MB
None
                review_id                 user_id             business_id  \
0  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
1  JrIxlS1TzJ-iCu79ul40cQ  eUta8W_HdHMXPzLBBZhL1A  04UD14gamNjLY0IDYVhHJg   

   stars  useful           date sentiment  \
0      5       1  142032966300

In [13]:
import json
import pandas as pd
from tqdm import tqdm

# 1. JSONL 로드 함수
def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

# 2. 경로
review_path = "data/review_0503.json"
user_path = "data/user_0503.json"
business_path = "data/business.json"
output_path = "data/merged_dataset.json"

# 3. 로딩
reviews = load_jsonl(review_path)
users = load_jsonl(user_path)
businesses = load_jsonl(business_path)

# 4. 빠른 조회용 dict
user_dict = {u["user_id"]: u for u in users}
business_dict = {b["business_id"]: b for b in businesses}

# 5. 병합 (모든 필드 포함 + prefix 부여)
merged_data = []
skipped = 0

for r in tqdm(reviews, desc="병합 중"):
    uid = r.get("user_id")
    bid = r.get("business_id")

    u = user_dict.get(uid)
    b = business_dict.get(bid)

    if not u or not b:
        skipped += 1
        continue

    # 유효성 필터
    if not all([r.get("review_id"), uid, bid, r.get("cleaned_text"), r.get("sentiment")]):
        skipped += 1
        continue

    merged_entry = {}

    # 리뷰 필드
    merged_entry.update(r)

    # user_ prefix 붙여서 사용자 필드 추가
    for k, v in u.items():
        merged_entry[f"user_{k}"] = v

    # business_ prefix 붙여서 음식점 필드 추가
    for k, v in b.items():
        merged_entry[f"business_{k}"] = v

    merged_data.append(merged_entry)

# 6. 저장 (JSONL)
with open(output_path, "w", encoding="utf-8") as f:
    for row in merged_data:
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

# 7. 확인
print(f"✅ 병합 완료: {output_path}")
print(f"총 병합 리뷰 수: {len(merged_data)}")
print(f"제외된 리뷰 수: {skipped}")
print(f"최종 컬럼 수: {len(merged_entry)}")
print("예시 컬럼:", list(merged_entry.keys())[:10], "...")


병합 중: 100%|██████████| 676701/676701 [00:02<00:00, 323813.05it/s]


✅ 병합 완료: data/merged_dataset.json
총 병합 리뷰 수: 404121
제외된 리뷰 수: 272580
최종 컬럼 수: 23
예시 컬럼: ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'date', 'sentiment', 'cleaned_text', 'token_length', 'user_user_id'] ...


In [14]:
from collections import Counter

# 1. user_id별 리뷰 수 계산
user_review_counts = Counter([row["user_id"] for row in merged_data])

# 2. 필터링: 리뷰 5개 이상만 포함
filtered_data = [
    row for row in merged_data
    if user_review_counts[row["user_id"]] >= 5
]

# 3. 저장
filtered_output_path = "data/merged_dataset.json"
with open(filtered_output_path, "w", encoding="utf-8") as f:
    for row in filtered_data:
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

# 4. 통계 출력
print("\n✅ 필터링 완료: 리뷰 5개 미만 유저 제외")
print(f"- 총 유저 수: {len(set(user_review_counts))}")
print(f"- 리뷰 5개 미만 유저 수: {sum(1 for v in user_review_counts.values() if v < 5)}")
print(f"- 최종 포함 리뷰 수: {len(filtered_data)}")


✅ 필터링 완료: 리뷰 5개 미만 유저 제외
- 총 유저 수: 26894
- 리뷰 5개 미만 유저 수: 0
- 최종 포함 리뷰 수: 404121


In [15]:
import json
import pandas as pd

# 1. JSONL 파일 경로
path = "data/merged_dataset.json"  # 필요 시 경로 수정

# 2. JSONL 파일 읽기
with open(path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# 3. pandas DataFrame으로 변환
df = pd.DataFrame(data)

# 4. 기본 정보 출력
print("📊 [기본 정보]")
print(df.info())
print()

# 5. 컬럼 목록
print("🧾 [컬럼 목록]")
print(df.columns.tolist())
print()

# 6. 예시 데이터
print("🔍 [샘플 데이터]")
print(df.head(3).to_string(index=False))


📊 [기본 정보]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404121 entries, 0 to 404120
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   review_id              404121 non-null  object 
 1   user_id                404121 non-null  object 
 2   business_id            404121 non-null  object 
 3   stars                  404121 non-null  int64  
 4   useful                 404121 non-null  int64  
 5   date                   404121 non-null  int64  
 6   sentiment              404121 non-null  object 
 7   cleaned_text           404121 non-null  object 
 8   token_length           404121 non-null  int64  
 9   user_user_id           404121 non-null  object 
 10  user_name              404121 non-null  object 
 11  user_review_count      404121 non-null  int64  
 12  user_useful            404121 non-null  int64  
 13  user_average_stars     404121 non-null  float64
 14  business_business_id   404

In [ ]:
import json

# 3. 중복/불필요한 컬럼 제거
drop_cols = ["user_user_id", "business_business_id"]
df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)

# 4. 컬럼 이름 리네이밍 (명확하게 하기)
df.rename(columns={
    "stars": "review_stars",
    "useful": "review_useful",
    "date": "review_date"
}, inplace=True)

# 5. JSONL로 저장
output_path = "data/merged_dataset_cleaned.json"
with open(output_path, "w", encoding="utf-8") as f:
    for row in df.to_dict(orient="records"):
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

# 6. 요약 출력
print("✅ 정리 완료: JSONL 형식으로 저장")
print(f"저장 경로: {output_path}")
print("총 샘플 수:", len(df))
print("최종 컬럼 수:", len(df.columns))
print("샘플 컬럼:", df.columns[:10].tolist())

✅ 정리 완료: JSONL 형식으로 저장
저장 경로: data/merged_dataset_cleaned.json
총 샘플 수: 404121
최종 컬럼 수: 21
샘플 컬럼: ['review_id', 'user_id', 'business_id', 'review_stars', 'review_useful', 'review_date', 'sentiment', 'cleaned_text', 'token_length', 'user_name']


In [19]:
print(df.head(3).to_string(index=False))

             review_id                user_id            business_id  review_stars  review_useful   review_date sentiment                                                                                                                                                                                                  cleaned_text  token_length user_name  user_review_count  user_useful  user_average_stars     business_name business_address business_city business_state  business_stars  business_review_count  business_is_open                    business_categories
8JFGBuHMoiNDyfcxuWNtrA smOvOajNG0lS4Pq7d8g4JQ RZtGWDLCAtuipwaZ-UfjmQ             4              0 1255550234000  Positive                                                                                                                   good foodlove gnocchi marinara baked eggplant appetizer good service slow despite food good            20       Meg                176          120                3.46         LaScala's  615 Chestnut 